In [97]:
SQLContext.newSession(sqlContext)
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler,StandardScaler,RFormula
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.linalg import VectorUDT,Vectors
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.sql import Window
from pyspark.ml import Pipeline
from pyspark.ml.regression import GeneralizedLinearRegression

import pandas as pd
import re
from tabulate import tabulate
import random
import sys
import matplotlib.pyplot as plt
import numpy as np

In [39]:
#import data and rename bad name rank into vaerdiSlope
#RAW DATA!!! 


#exclude some of the variables, and cast all variables to double
excludeCols = ["medArb_"+str(i) for i in range(1,16)] # we don't need the medarbejders 
includeCols = [i for i in df.columns if i not in excludeCols]

rankCols = [re.sub(pattern="rank_",repl="vaerdiSlope_",string=i) for i in includeCols]
finalCols = [F.col(i) for i in includeCols[:2]]+["kortBeskrivelse"]+[F.col(i).cast("double") for i in includeCols[2:] if i not in ["kortBeskrivelse"]]


df = sqlContext.read.parquet("/home/svanhmic/workspace/Python/Erhvervs/data/cdata/featureDataCvr")
df.select(["cvrNummer"])
rankCols = [re.sub(pattern="rank_",repl="vaerdiSlope_",string=i) for i in includeCols ]
renamedDf = (df
             .select(*finalCols)
             .select([F.col(val).alias(rankCols[idx]) for idx,val in enumerate(includeCols)])
             .filter((F.col("kortBeskrivelse") == "APS") | (F.col("kortBeskrivelse") == "AS"))
             )
renamedDf.show()

+---------+--------------------+-----+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+-----------+-----------------+------------------+------------------+------------------+------------------+-------------+-------------+-------------+-------------+----------------+---------------+
|cvrNummer|              status|label|AarsVaerk_1|AarsVaerk_2|AarsVaerk_3|AarsVaerk_4|AarsVaerk_5|AarsVaerk_6|AarsVaerk_7|AarsVaerk_8|AarsVaerk_9|AarsVaerk_10|AarsVaerk_11|AarsVaerk_12|AarsVaerk_13|AarsVaerk_14|AarsVaerk_15|avgVarighed|totalAabneEnheder|totalLukketEnheder|     vaerdiSlope_1|     vaerdiSlope_2|     vaerdiSlope_3|vaerdiSlope_4|vaerdiSlope_5|vaerdiSlope_6|vaerdiSlope_7|reklamebeskyttet|kortBeskrivelse|
+---------+--------------------+-----+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+-

In [40]:
windowSpecRank =(Window.partitionBy(F.col("cvrNummer"))).orderBy(F.col("periode_gyldigFra").desc())
groupCols = ["cvrNummer","vaerdi"]

companyNameDf = (sqlContext
                 .read
                 .parquet("/home/svanhmic/workspace/Python/Erhvervs/data/cdata/companyCvrData")
                 .withColumn(colName="rank",col=F.rank().over(windowSpecRank))
                 .filter((F.col("rank")==1) & (F.col("sekvensnr")==0))
                 .select([F.col(i) for i in groupCols])
                 .withColumnRenamed(existing="vaerdi",new="navn")
                 .orderBy(F.col("cvrNummer"))
                 .cache()
                )

In [41]:
labelCols = ["navn","cvrNummer","label","status","kortBeskrivelse"]
featCols = [i for i in companyNameDf.columns+renamedDf.columns if i not in labelCols]

#get minimum values from each column
minCols = [F.min(i).alias(i) for i in featCols]
minValsRdd = renamedDf.groupby().agg(*minCols).rdd
broadcastedmin = sc.broadcast(minValsRdd.first().asDict())

#create array that subtracts minimum value in the numeric columns.
logColsSelected = [F.col(i).alias(i) for i in labelCols]+[(F.col(i)-F.lit(broadcastedmin.value[i])).alias(i) for i in featCols]

#takes log(x+1) to the numeric columns and fills the blanks with 0.0 
logDf = (renamedDf
         .join(companyNameDf,(companyNameDf["cvrNummer"]==renamedDf["cvrNummer"]),"inner")
         .drop(companyNameDf["cvrNummer"])
         .select(*logColsSelected)
         #.select([F.col(i).alias(i) for i in labelCols]+[F.log1p(F.col(i)).alias(i) for i in featCols])
         .distinct()
         .na
         .fill(0.0,featCols)
         .cache()
        )
logDf.show(4)

+--------------------+---------+-----+--------------------+---------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+-----------+-----------------+------------------+--------------------+-------------+-------------+-------------+-------------+-------------+-------------+----------------+
|                navn|cvrNummer|label|              status|kortBeskrivelse|AarsVaerk_1|AarsVaerk_2|AarsVaerk_3|AarsVaerk_4|AarsVaerk_5|AarsVaerk_6|AarsVaerk_7|AarsVaerk_8|AarsVaerk_9|AarsVaerk_10|AarsVaerk_11|AarsVaerk_12|AarsVaerk_13|AarsVaerk_14|AarsVaerk_15|avgVarighed|totalAabneEnheder|totalLukketEnheder|       vaerdiSlope_1|vaerdiSlope_2|vaerdiSlope_3|vaerdiSlope_4|vaerdiSlope_5|vaerdiSlope_6|vaerdiSlope_7|reklamebeskyttet|
+--------------------+---------+-----+--------------------+---------------+-----------+-----------+-----------+-----------+-----------

In [56]:
strs = ""
excludedCols = ["cvrNummer","label","status","navn","kortBeskrivelse"]
for i in logDf.columns:
    if i not in excludedCols:
        strs += i+" + "

#excludedCols    
imputedDf = logDf.fillna(value=0.0)
formula = RFormula(formula="label ~ "+strs[:-3],labelCol="label")

glr = GeneralizedLinearRegression(family="binomial", link="logit", maxIter=10, regParam=0.3)
standardScale = StandardScaler(withMean=True,withStd=True,inputCol=glr.getFeaturesCol(),outputCol="scaledFeatures")


pipeline = Pipeline(stages=[formula,standardScale,glr])

grid = (ParamGridBuilder()
        .baseOn({lr.predictionCol:"prediction"})
        .baseOn({lr.rawPredictionCol:"rawPrediction"})
        .baseOn({lr.probabilityCol:"probability"})
        .baseOn({lr.labelCol:"label"})
        .baseOn({lr.featuresCol:"features"})
        .addGrid(param=lr.elasticNetParam,values=[0.1,1.0])
        .addGrid(param=lr.getMaxIter,values=[10])
        .build()
       )

evaluate = BinaryClassificationEvaluator()

trainEvalModel = TrainValidationSplit(estimator=pipeline,estimatorParamMaps=grid,evaluator=evaluate,trainRatio=0.8)

In [71]:
cols = [i for i in logDf.columns if i not in excludedCols]+["label"]

model = pipeline.fit(imputedDf.filter(F.col("label") <= 1).select(*cols))

In [80]:
predict = model.transform(imputedDf.select(*cols).filter(F.col("label") <= 1))
coef = model.stages[-1]


DenseVector([-0.0302, 0.0647, -0.0222, 0.0033, -0.0118, -0.0069, -0.0114, -0.0141, -0.0237, 0.0026, 0.0059, -0.0114, 0.0228, -0.0217, -0.0318, 0.0001, 0.3422, -0.7792, -0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, -0.1246])

In [59]:
p = model.stages[-1].summary

print("Coefficient Standard Errors: " + str(p.coefficientStandardErrors))
print("T Values: " + str(p.tValues))
print("P Values: " + str(p.pValues))
print("Dispersion: " + str(p.dispersion))
print("Null Deviance: " + str(p.nullDeviance))
print("Residual Degree Of Freedom Null: " + str(p.residualDegreeOfFreedomNull))
print("Deviance: " + str(p.deviance))
print("Residual Degree Of Freedom: " + str(p.residualDegreeOfFreedom))
print("AIC: " + str(p.aic))
print("Deviance Residuals: ")
p.residuals().show()

Coefficient Standard Errors: [0.002715365786167704, 0.0033576183070386156, 0.0036618804887292387, 0.003575453149894902, 0.0048004424800104665, 0.005307034626833281, 0.006044648209200046, 0.006661858012195801, 0.007406240336102588, 0.007567900680939736, 0.008093150133059149, 0.008512938676244104, 0.008348498511501061, 0.009076922598213254, 0.009345658737006966, 3.570481466497788e-06, 0.007713032831503935, 0.011306086099506183, 2.216392818255699e-10, 4.2738460918044474e-08, 1.7226360583964903e-08, 1.5410312395327645e-09, 1.1484656996653036e-07, 1.2769605473113263e-07, 3.590073080014067e-07, 0.012775368232153422, 0.01384685535599953]
T Values: [-11.13953124016808, 19.273340427415835, -6.059268404920775, 0.9099659049882868, -2.4512213897007147, -1.309316768159207, -1.891499436869883, -2.1100010055623186, -3.1957062897626076, 0.34554268948276695, 0.7313830924481778, -1.3424645212472799, 2.734757516096776, -2.3952850139227797, -3.3976150832275978, 19.491283138791598, 44.3648814342137, -68.92

In [105]:
print(len(cols))
print(type(coef.coefficients.toArray()))
print()
      
      
summary = {"Labels":cols[:-1]+["intercept"],"Coefficients":np.insert(coef.coefficients.toArray(),0,coef.intercept),"coefficient Std Err":p.coefficientStandardErrors,"T Values":p.tValues,"P Values":p.pValues}

27
<class 'numpy.ndarray'>



In [116]:
pd.options.display.float_format = '{:,.4f}'.format
df = pd.DataFrame(summary,columns=["Labels","Coefficients","coefficient Std Err","T Values","P Values"])
import subprocess

HEADER = '''
<html>
    <head>
        <style>
            .df tbody 
        </style>
    </head>
    <body>
'''
FOOTER = '''
    </body>
</html>
'''

#df = pd.DataFrame({'a': np.arange(10), 'b': np.random.randn(10)})
with open('test.html', 'w') as f:
    f.write(HEADER)
    f.write(df.to_html(classes='df'))
    f.write(FOOTER)

In [122]:
#check mean and stddev
df
#descriptionCVR.filter((F.col("summary") =="mean") | (F.col("summary") =="stddev")).show()

,Labels,Coefficients,coefficient Std Err,T Values,P Values
0,AarsVaerk_1,-1.2061,0.0027,-11.1395,0.0000
1,AarsVaerk_2,-0.0302,0.0034,19.2733,0.0000
2,AarsVaerk_3,0.0647,0.0037,-6.0593,0.0000
3,AarsVaerk_4,-0.0222,0.0036,0.9100,0.3628
4,AarsVaerk_5,0.0033,0.0048,-2.4512,0.0142
5,AarsVaerk_6,-0.0118,0.0053,-1.3093,0.1904
6,AarsVaerk_7,-0.0069,0.0060,-1.8915,0.0586
7,AarsVaerk_8,-0.0114,0.0067,-2.1100,0.0349
8,AarsVaerk_9,-0.0141,0.0074,-3.1957,0.0014
9,AarsVaerk_10,-0.0237,0.0076,0.3455,0.7297


In [119]:
windowSpecRank =(Window.partitionBy(F.col("cvrNummer"))).orderBy(F.col("periode_gyldigFra").desc())

groupCols = ["cvrNummer","vaerdi"]

companyNameDf = (sqlContext
                 .read
                 .parquet("/home/svanhmic/workspace/Python/Erhvervs/data/cdata/"+"companyCvrData")
                 .withColumn(colName="rank",col=F.rank().over(windowSpecRank))
                 .filter((F.col("rank")==1) & (F.col("sekvensnr")==0))
                 .select([F.col(i) for i in groupCols])
                 .withColumnRenamed(existing="vaerdi",new="navn")
                 .orderBy(F.col("cvrNummer"))
                 .cache()
                )
companyNameDf.show(2)

+---------+--------------------+
|cvrNummer|                navn|
+---------+--------------------+
| 10000009|              YELLOW|
| 10000025|WATERFRONT CONNEC...|
+---------+--------------------+
only showing top 2 rows



In [128]:
logDf.show()

+--------------------+---------+-----+--------------------+---------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+------------+------------+-----------+-----------------+------------------+--------------------+-----------------+------------------+-------------+-------------+-------------+-------------+----------------+
|                navn|cvrNummer|label|              status|kortBeskrivelse|AarsVaerk_1|AarsVaerk_2|AarsVaerk_3|AarsVaerk_4|AarsVaerk_5|AarsVaerk_6|AarsVaerk_7|AarsVaerk_8|AarsVaerk_9|AarsVaerk_10|AarsVaerk_11|AarsVaerk_12|AarsVaerk_13|AarsVaerk_14|AarsVaerk_15|avgVarighed|totalAabneEnheder|totalLukketEnheder|       vaerdiSlope_1|    vaerdiSlope_2|     vaerdiSlope_3|vaerdiSlope_4|vaerdiSlope_5|vaerdiSlope_6|vaerdiSlope_7|reklamebeskyttet|
+--------------------+---------+-----+--------------------+---------------+-----------+-----------+-----------+-----

In [131]:
#take ln(x+1) of features
#First convert features to vetor
toDenseUDf = F.udf(lambda x: Vectors.dense(x.toArray()),VectorUDT())
vectorizer = VectorAssembler(inputCols=logFeatCols,outputCol="features")

labelCols = ["cvrNummer","label","status","kortBeskrivelse"]
logFeatCols = [i for i in logDf.columns if i not in labelCols]
#print(logFeatCols)

rawVectorDataDf = (vectorizer.transform(logDf)
                   .select(["navn"]+labelCols+[toDenseUDf(vectorizer.getOutputCol()).alias(vectorizer.getOutputCol())])
                  )

standardScale = StandardScaler(withMean=True,withStd=True,inputCol=vectorizer.getOutputCol(),outputCol="scaledFeatures")
standardScaleModel = standardScale.fit(rawVectorDataDf)
scaledFeaturesDf = (standardScaleModel
                    .transform(rawVectorDataDf)
                    .drop("features")
                    .withColumnRenamed(existing="scaledFeatures",new="features")
                   )

scaledFeaturesDf.show()

+--------------------+---------+-----+--------------------+---------------+--------------------+
|                navn|cvrNummer|label|              status|kortBeskrivelse|            features|
+--------------------+---------+-----+--------------------+---------------+--------------------+
|KØBMANDSSELSKABET...| 10033764|  0.0|            [NORMAL]|            APS|[0.21869966056626...|
|JB PROFESSIONEL B...| 10045126|  1.0|[OPLØST EFTER KON...|            APS|[-0.1557084560789...|
|AGENDA MANAGEMENT...| 10052262|  1.0|[OPLØST EFTER KON...|            APS|[-0.1557084560789...|
|TORNUM SNEDKER- O...| 10059968|  0.0|            [NORMAL]|            APS|[-0.0621064269176...|
|     RESEARCHGRUPPEN| 10069122|  0.0|            [NORMAL]|            APS|[-0.0621064269176...|
|            WELLWARE| 10085357|  1.0|[OPLØST EFTER KON...|            APS|[0.21869966056626...|
|     EJUP ZEKIROVSKI| 10093767|  1.0|[OPLØST EFTER KON...|            APS|[-0.1557084560789...|
|  PROJEKT DEVELOPING| 1010722

In [132]:
#put them into a feature vecto
vectorizedTestDf = scaledFeaturesDf.filter(F.col("label") <= 1).sampleBy("label", fractions={0: 0.2, 1: 0.2}, seed=42)
vectorizedTestDf.groupBy("label").count().show()

scaledCvrDf = scaledFeaturesDf.select(F.col("cvrNummer"))
cvrTestDf = vectorizedTestDf.select("cvrNummer")
cvrTrainDf = scaledCvrDf.subtract(cvrTestDf) #take the other partion as training set

vectorizedTrainDf = (scaledFeaturesDf
                     .filter(F.col("label") <= 1)
                     .join(cvrTrainDf,(scaledFeaturesDf["cvrNummer"] == cvrTrainDf["cvrNummer"]),"inner")
                     .drop(cvrTrainDf["cvrNummer"])
                    )
vectorizedTrainDf.groupBy("label").count().show()
print("Number of data points: "+str(scaledFeaturesDf.count()))
print("Number of data points train: "+str(vectorizedTrainDf.select("cvrNummer").count()))
print("Number of data points test: "+str(vectorizedTestDf.select("cvrNummer").count()))
#vectorizedTrainDf.printSchema()
#print(vectorizedTrainDf.first())

+-----+-----+
|label|count|
+-----+-----+
|  0.0|19107|
|  1.0| 4873|
+-----+-----+

+-----+-----+
|label|count|
+-----+-----+
|  0.0|76822|
|  1.0|19436|
+-----+-----+

Number of data points: 120814
Number of data points train: 96258
Number of data points test: 23980


In [133]:
vectorizedTrainDf.show()

+--------------------+-----+--------------------+---------------+--------------------+---------+
|                navn|label|              status|kortBeskrivelse|            features|cvrNummer|
+--------------------+-----+--------------------+---------------+--------------------+---------+
|KØBMANDSSELSKABET...|  0.0|            [NORMAL]|            APS|[0.21869966056626...| 10033764|
|JB PROFESSIONEL B...|  1.0|[OPLØST EFTER KON...|            APS|[-0.1557084560789...| 10045126|
|AGENDA MANAGEMENT...|  1.0|[OPLØST EFTER KON...|            APS|[-0.1557084560789...| 10052262|
|TORNUM SNEDKER- O...|  0.0|            [NORMAL]|            APS|[-0.0621064269176...| 10059968|
|     RESEARCHGRUPPEN|  0.0|            [NORMAL]|            APS|[-0.0621064269176...| 10069122|
|            WELLWARE|  1.0|[OPLØST EFTER KON...|            APS|[0.21869966056626...| 10085357|
|UNIQUE SERVICE PA...|  0.0|            [NORMAL]|            APS|[-0.1557084560789...| 10133459|
|LAURA & VURAL INT...|  0.0|[T

In [134]:
#Train the logistic regressionmodel
lr = LogisticRegression()
grid = (ParamGridBuilder()
        .baseOn({lr.predictionCol:"prediction"})
        .baseOn({lr.rawPredictionCol:"rawPrediction"})
        .baseOn({lr.probabilityCol:"probability"})
        .baseOn({lr.labelCol:"label"})
        .baseOn({lr.featuresCol:"features"})
        .addGrid(param=lr.elasticNetParam,values=[0.1,1.0])
        .addGrid(param=lr.getMaxIter,values=[10])
        .build()
       )
evaluate = BinaryClassificationEvaluator(rawPredictionCol="rawPrediction")
crossVal = CrossValidator(estimator=lr,estimatorParamMaps=grid,evaluator=evaluate,numFolds=10)

crossValModel = crossVal.fit(dataset=vectorizedTrainDf)
evaluate.evaluate(crossValModel.transform(vectorizedTestDf))
#coef = lrModel.coefficients

0.8119230817933306

In [135]:
bestModel = crossValModel.bestModel

In [136]:
#test the values
result = bestModel.transform(vectorizedTestDf)

In [12]:
#

In [137]:
#result.orderBy("prediction").show(100)
confCols = [F.col(i) for i in ["TP","TN","FP","FN"]]


csCols = [F.when((F.col("label")==1) & (F.col("difference") == 0),"TP")
          ,F.when((F.col("label")==0) & (F.col("difference") == 0),"TN")
          ,F.when(F.col("difference") == 1,"FN")
          ,F.when(F.col("difference") == -1,"FP")
         ]

confusionDf = result.select(F.col("label"),F.col("prediction"),(F.col("label")-F.col("prediction")).alias("difference"))
(confusionDf
 .select(F.coalesce(*csCols).alias("cases")  
         #,.otherwise(0).alias("FP")
         #,.otherwise(0).alias("FN")
        )
 .groupBy("cases").count()
).show()

 

+-----+-----+
|cases|count|
+-----+-----+
|   TP|  329|
|   TN|18920|
|   FN| 4544|
|   FP|  187|
+-----+-----+



In [164]:
acc = confusionDf.groupBy("cases").max().take(1)
print(acc)

AnalysisException: "cannot resolve '`cases`' given input columns: [label, prediction, difference];"

In [144]:
crossValModel.bestModel.hasSummary

True

In [145]:
summary = crossValModel.bestModel.summary

In [163]:
summary.areaUnderROC
summary.fMeasureByThreshold.show()
summary.pr.show()
summary.precisionByThreshold.show()
summary.roc.show()
summary.recallByThreshold.show()
summary.totalIterations
summary.

#for s in summary:
#    print(s + str(summary[s])+"\n")

+-------------------+--------------------+
|          threshold|           F-Measure|
+-------------------+--------------------+
| 0.9999999998262108|0.018052738336713996|
| 0.6029720467816753| 0.04342845759250959|
| 0.5548895764816145|0.058415695530314206|
| 0.5352922194681604| 0.07590887057682986|
| 0.5226281290191139| 0.09642227621361536|
| 0.5104669745067404| 0.11305530797016466|
| 0.5027036316785367| 0.12816842882866245|
|0.49702833276414865|   0.140203932993445|
|0.49345764045064777| 0.15536496024300903|
|0.48963009953953224|  0.1670701113507328|
|  0.487418026345812| 0.17703100640914604|
| 0.4851876507384984| 0.18672464509528072|
| 0.4830943564912248| 0.19793452356064722|
| 0.4811206606975969| 0.21139675781889636|
| 0.4792802930278031| 0.22196289887730875|
| 0.4778307111343987|  0.2357077950298289|
|0.47643159133543633| 0.24835232999922463|
| 0.4750421306102541|  0.2579611761565158|
| 0.4739300653336836|  0.2701529944263644|
| 0.4725752166604857| 0.27834291611381395|
+----------

46

In [24]:
summary.predictions.show()

+--------------------+-----+--------------------+--------------------+---------+--------------------+--------------------+----------+
|                navn|label|              status|            features|cvrNummer|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+---------+--------------------+--------------------+----------+
|SKYTTENS HANDEL O...|    1|[OPLØST EFTER FUS...|[-0.1483813281958...| 10019052|[-0.1436615979228...|[0.46414624375980...|       1.0|
|            DIKI.NET|    0|[OPLØST EFTER KON...|[-0.1483813281958...| 10026113|[-0.2470744652013...|[0.43854370342953...|       1.0|
|                CTEK|    1|[OPLØST EFTER FUS...|[-0.1177596186074...| 10040523|[-0.6803097893868...|[0.33619216432178...|       1.0|
|      VG ENTREPRENØR|    1|            [NORMAL]|[0.43343115398404...| 10057426|[-3.7227933151943...|[0.02359613625108...|       1.0|
|NORDBYENS OLIEFYR...|    1|            [NORMAL]|[-0.148381328